In [21]:
!nvidia-smi

Sun Jun 23 19:28:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              44W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [11]:
!pip install roboflow torch torchvision torchaudio ultralytics python-dotenv --quiet

In [20]:
import os
import torch
from roboflow import Roboflow
from ultralytics import YOLO
from pathlib import Path
from google.colab import drive
from dotenv import load_dotenv

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
ROOT_DIR = Path('/content/drive/MyDrive/tennis-cv/').resolve()

In [15]:
TRAINING_DIR = ROOT_DIR / 'training'
MODELS_DIR = ROOT_DIR / 'models'

DATA_DIR = ROOT_DIR / 'data'
TENNIS_BALL_DIR = DATA_DIR / 'tennis_ball'
KEYPOINTS_DIR = DATA_DIR / 'keypoints'

DATA_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

TRAINING_DIR.mkdir(parents=True, exist_ok=True)
TENNIS_BALL_DIR.mkdir(parents=True, exist_ok=True)
KEYPOINTS_DIR.mkdir(parents=True, exist_ok=True)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [19]:
if not any(TENNIS_BALL_DIR.iterdir()):
    print(f"No data found in {TENNIS_BALL_DIR}, downloading dataset...")
    load_dotenv(ROOT_DIR / '.env')
    rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
    project = rf.workspace("ry4ntr1").project("tennis-ball-m7zvw")
    version = project.version(3)
    dataset = version.download(model_format="yolov8", location=str(TENNIS_BALL_DIR), overwrite=True)
else:
    print(f"Data already present in {TENNIS_BALL_DIR}, no need to download.")

No data found in /content/drive/MyDrive/tennis-cv/data/tennis_ball, downloading dataset...
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.40, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to /content/drive/MyDrive/tennis-cv/data/tennis_ball in yolov8:: 100%|██████████| 6785/6785 [01:09<00:00, 98.27it/s] 


In [28]:
os.chdir(str(ROOT_DIR))

In [25]:
if os.path.exists(str(MODELS_DIR / 'best.pt')):
    trained_model = YOLO(str(MODELS_DIR / 'best.pt'))
else:
    model = YOLO('yolov8n.pt')
    model.train(data=str(TENNIS_BALL_DIR / 'data.yaml'), save=True, save_dir=MODELS_DIR, save_period=10, verbose=True)

New https://pypi.org/project/ultralytics/8.2.41 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.40 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/tennis-cv/data/tennis_ball/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, ret

train: Scanning /content/drive/MyDrive/tennis-cv/data/tennis_ball/train/labels.cache... 2371 images, 10 backgrounds, 0 corrupt: 100%|██████████| 2371/2371 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/tennis-cv/data/tennis_ball/train/images/scene57901_jpg.rf.6511105d25878ddf4e7d66b7b62ce3f1.jpg: 4 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/tennis-cv/data/tennis_ball/train/images/scene57901_jpg.rf.afaa87d778ed1646b6b88870dc8e70f5.jpg: 4 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


SystemError: <built-in function _error_if_any_worker_fails> returned a result with an exception set

In [29]:
trained_model = YOLO('/content/runs/detect/train/weights/best.pt').to(device)
# trained_model = YOLO(str(MODELS_DIR / 'best.pt')).to(device)
sample_image_path = ROOT_DIR / 'sample_data/sample.png'
sample_vid_path = ROOT_DIR / 'sample_data/sample.mp4'

## Prediction On Sample Data

In [31]:
trained_model.predict(task='detect', source=str(sample_image_path), save=True, save_dir=str('/content/runs/predict'))


image 1/1 /content/drive/MyDrive/tennis-cv/sample_data/sample.png: 320x640 2 playerss, 1 tennis_ball, 11.3ms
Speed: 3.2ms preprocess, 11.3ms inference, 2.2ms postprocess per image at shape (1, 3, 320, 640)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'official', 1: 'players', 2: 'tennis_ball'}
 obb: None
 orig_img: array([[[ 24,  23,  38],
         [ 24,  23,  38],
         [ 24,  23,  38],
         ...,
         [ 30,  24,  42],
         [ 30,  24,  42],
         [ 20,  20,  20]],
 
        [[ 24,  23,  38],
         [ 24,  23,  38],
         [ 24,  23,  38],
         ...,
         [ 31,  24,  42],
         [ 31,  24,  42],
         [ 20,  20,  20]],
 
        [[ 24,  23,  38],
         [ 24,  23,  38],
         [ 24,  23,  38],
         ...,
         [ 31,  24,  42],
         [ 31,  24,  42],
         [ 20,  20,  20]],
 
        ...,
 
        [[187, 153, 113],
         [187, 153, 113],
         [187, 153, 113],
         ...,
         [178, 145, 107],
         [178, 145, 107],
         [  0,   0,   0]],
 
        [[187, 153, 113],
         [187, 153, 113],
         [187, 153, 113]

In [37]:
trained_model.predict(task='detect', source=str(sample_vid_path), save=True, save_dir=str('/content/runs/predict'))



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/214) /content/drive/MyDrive/tennis-cv/sample_data/sample.mp4: 384x640 1 official, 3 playerss, 1 tennis_ball, 121.1ms
video 1/1 (frame 2/214) /content/drive/MyDrive/tennis-cv/sample_data/sample.mp4: 384x640 1 official, 3 playerss, 1 tennis_ball, 8.5ms
video 1/1 (frame 3/214) /content/drive/MyDrive/tennis-cv/sample_data/sample.mp4: 384x640 1 official, 3 playerss, 2 tennis_balls, 7.8ms
video 1/1 (frame 4/214) /content/drive/MyDrive/tenn

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'official', 1: 'players', 2: 'tennis_ball'}
 obb: None
 orig_img: array([[[ 20,  12,  25],
         [ 19,  11,  24],
         [ 18,  10,  23],
         ...,
         [ 13,   0,  36],
         [ 15,   2,  38],
         [ 16,   3,  39]],
 
        [[ 20,  12,  25],
         [ 20,  12,  25],
         [ 19,  11,  24],
         ...,
         [ 22,   9,  45],
         [ 23,  10,  46],
         [ 24,  11,  47]],
 
        [[ 21,  13,  28],
         [ 21,  13,  28],
         [ 20,  11,  29],
         ...,
         [ 32,  18,  57],
         [ 35,  18,  58],
         [ 36,  19,  59]],
 
        ...,
 
        [[182, 149, 111],
         [182, 149, 111],
         [182, 149, 111],
         ...,
         [174, 143, 105],
         [174, 143, 105],
         [174, 143, 105]],
 
        [[182, 149, 111],
         [182, 149, 111],
         [182, 149, 111]

## Export To Core ML

In [ ]:
model.export(format='coreml', save=True)